In [ ]:
#!/usr/bin/env python3
"""
run_dnn_iv_vs_yields.py  ──  imp_vol+yields vs yields-only 비교
===========================================================

* YIELDS      : fwds 단일-브랜치 DNN
* IV+YIELDS   : imp_vol vs fwds 듀얼-브랜치 DNN_DUAL
"""
import os, sys, argparse, warnings
import pandas as pd
warnings.filterwarnings("ignore")

from rolling_framework import Machine

# ─────────────────────────── USER CONFIG ───────────────────────────
DATA_ROOT     = "./data/19712023"
OUTPUT_ROOT   = "./output"
OUT_DEFAULT   = "results_iv_vs_yields.csv"

BURN_START    = "199009"
BURN_END      = "200609"
PERIOD_START  = "199009"
PERIOD_END    = "202212"
HORIZON       = 12  # months ahead

MATURITIES    = ["xr_2","xr_3","xr_5","xr_7","xr_10"]

# DNN 하이퍼-파라미터 (둘 모두 동일)
param_grid_dnn  = {
    "dnn__module__hidden"          : [(3,)],
    "dnn__module__dropout"         : [0.1],
    "dnn__optimizer__lr"           : [1e-3],
    "dnn__optimizer__weight_decay" : [1e-4],
}
param_grid_dual = {
    # branch1 (imp_vol) 구조
    "dnn__module__hidden1"         : [(1,)],
    "dnn__module__drop1"           : [0.1],

    # branch2 (fwds) 구조
    "dnn__module__hidden2"         : [(16,),],
    "dnn__module__drop2"           : [0.1],

    # merge 방식 (concat/sum/gate) 도 탐색 가능
    "dnn__module__merge"           : ["gate"],

    # optimizer
    "dnn__optimizer__lr"           : [1e-3],
    "dnn__optimizer__weight_decay" : [1e-4],

    # branch별 학습률 튜닝 (SafeNetDualLR 사용 시)
    "dnn__lr_br1"                  : [5e-4],
    "dnn__lr_br2"                  : [1e-2],
    "dnn__lr_head"                 : [1e-3],
}
def main():
    # Jupyter/VSCode 런처가 붙이는 --f 등 무시
    p = argparse.ArgumentParser(add_help=False)
    p.add_argument("--out", default=None, help="출력 CSV 경로")
    args, _ = p.parse_known_args()

    os.makedirs(OUTPUT_ROOT, exist_ok=True)
    out_csv = args.out or os.path.join(OUTPUT_ROOT, OUT_DEFAULT)

    # ─── 1) 데이터 로드 ────────────────────────────────────────────
    try:
        y   = pd.read_csv(f"{DATA_ROOT}/exrets_19712023.csv", index_col="Time")
        fw  = pd.read_csv(f"{DATA_ROOT}/fwds_19712023.csv",   index_col="Time")
        iv  = pd.read_csv(f"data/19902022/imp_vol_19902022.csv",index_col="Time")
        yl = pd.read_csv(f"data/19902022/yl_all_19902022.csv", index_col='Time')
    except FileNotFoundError as e:
        sys.exit(f"[ERROR] CSV not found → {e.filename}")

    # 실제로 평가할 만기만 골라서
    cols = [c for c in MATURITIES if c in y.columns]
    if not cols:
        sys.exit(f"[ERROR] None of {MATURITIES} in exrets")
    y = y[cols]

    # ─── 2) 피처 블록 생성 ──────────────────────────────────────────
    X_yields   = yl.copy()
    X_ivy      = pd.concat([iv, yl], axis=1)

    # ─── 3) 학습 & OOS-R2 수집 ───────────────────────────────────────
    rows = []

    
    # (A) imp_vol + yields → 듀얼-브랜치 DNN_DUAL
    iv_cols = iv.columns.tolist()
    yl_cols = yl.columns.tolist()
    m_ivy = Machine(
        X_ivy, y, "DNN_DUAL",
        option         = {"grp1": iv_cols, "grp2": yl_cols},
        params_grid    = param_grid_dual,
        burn_in_start  = BURN_START,
        burn_in_end    = BURN_END,
        period         = [PERIOD_START, PERIOD_END],
        forecast_horizon = HORIZON,
    )
    m_ivy.training()
    r2_ivy = m_ivy.R2OOS()
    for mty in cols:
        rows.append({
            "set"      : "IV+YIELDS",
            "maturity" : mty,
            "model"    : "DNN_DUAL_IV",
            "R2_OOS"   : float(r2_ivy[mty]),
        })
    
    # (B) YIELDS-only → 단일 DNN
    m_y = Machine(
        X_yields, y, "DNN",
        option         = None,
        params_grid    = param_grid_dnn,
        burn_in_start  = BURN_START,
        burn_in_end    = BURN_END,
        period         = [PERIOD_START, PERIOD_END],
        forecast_horizon = HORIZON,
    )
    m_y.training()
    r2_y = m_y.R2OOS()
    for mty in cols:
        rows.append({
            "set"      : "YIELDS",
            "maturity" : mty,
            "model"    : "DNN",
            "R2_OOS"   : float(r2_y[mty]),
        })


    # ─── 4) 저장 ───────────────────────────────────────────────────────
    pd.DataFrame(rows).to_csv(out_csv, index=False)
    print(f"★ Saved results → {out_csv}")


if __name__ == "__main__":
    main()

DNN_DUAL rolling:   4%|▍         | 20/520 [03:47<1:34:56, 11.39s/it]


KeyboardInterrupt: 